# Exercise 6

In [6]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
from scipy.stats import chi2_contingency, stats

In [7]:
df = pd.read_csv ('EFIplus_medit.zip',compression="zip", sep= ";")
df

,Site_code,Latitude,Longitude,Country,Catchment_name,Galiza,Subsample,Calib_EFI_Medit,Calib_connect,Calib_hydrol,...,Squalius malacitanus,Squalius pyrenaicus,Squalius torgalensis,Thymallus thymallus,Tinca tinca,Zingel asper,Squalius sp,Barbatula sp,Phoxinus sp,Iberochondrostoma_sp
0,ES_01_0002,38.102003,-4.096070,Spain,Guadalquivir,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,ES_02_0001,40.530188,-1.887796,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,ES_02_0002,40.595432,-1.928079,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,ES_02_0003,40.656184,-1.989831,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,ES_02_0004,40.676402,-2.036274,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5006,PTVO60,40.641612,-8.325149,Portugal,Vouga,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5007,PTVO61,40.729015,-8.098851,Portugal,Vouga,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5008,PTVO62,40.727959,-8.107344,Portugal,Vouga,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5009,PTVO63,40.817783,-8.224903,Portugal,Vouga,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Ex 1:

Using the EFIplus_medit.zip dataset, test if the frequency of sites with presence and absence of Salmo
trutta fario (Brown Trout) are independent from the country. Please state which is/are the null
hypothesis of your test(s). You may try to produce an alluvial plot.

In [8]:
df['Salmo_trutta_fario_presence'] = df['Salmo trutta fario'].apply(lambda x: 1 if x > 0 else 0)

In [9]:
contingency_table = pd.crosstab(df['Country'], df['Salmo_trutta_fario_presence'])
contingency_table

Salmo_trutta_fario_presence,0,1
Country,,
France,13,59
Italy,109,76
Portugal,615,252
Spain,1239,2648


In [10]:
# Perform Chi-square test
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

print(f"\nChi-Square Statistic: {chi2_stat:.2f}")
print(f"Degrees of Freedom: {dof}")
print(f"P-value: {p_val:.4f}")

if p_val < 0.05:
    print("Result: Reject the null hypothesis — species presence depends on country.")
else:
    print("Result: Fail to reject the null hypothesis — there is no evidence that species presence is dependent of country.")


Chi-Square Statistic: 496.37
Degrees of Freedom: 3
P-value: 0.0000
Result: Reject the null hypothesis — species presence depends on country.


### Ex 2:

Run the non-parametric equivalent of the test you used in exercise 5.3 and compare with the ANOVA
test (5.2: Test whether there are differences in the mean elevation in the upstream catchment
(Elevation_mean_catch) among the eight most sampled catchments. For which pairs of catchments are
these diferences significant? Please state which is/are the null hypothesis of your test(s)).


In [11]:

top8 = df['Catchment_name'].value_counts().head(8).index
df_clean = df[['Elevation_mean_catch', 'Catchment_name']].dropna()
df_top_8 = df_clean[df_clean['Catchment_name'].isin(top8)].copy()

df_top_8

,Elevation_mean_catch,Catchment_name
1,1603.519424,Tejo
2,1578.678579,Tejo
3,1553.219128,Tejo
4,1539.684999,Tejo
5,1493.914010,Tejo
...,...,...
4947,704.675069,Tejo
4948,702.673201,Tejo
4949,697.730192,Tejo
4950,679.784324,Tejo


In [12]:
# Prepare data for Kruskal-Wallis: group elevations by catchment
groups = [df_top_8['Elevation_mean_catch'][df_top_8['Catchment_name'] == cat] for cat in top8]

# Perform ANOVA test
f_stat, p_value = stats.f_oneway(*groups)

print(f"ANOVA F-statistic: {f_stat:.3f}, p-value: {p_value:.4f}")

# Perform Kruskal-Wallis test
h_stat, p_kw = stats.kruskal(*groups)

print(f"Kruskal-Wallis H-statistic: {h_stat:.3f}, p-value: {p_kw:.4f}")

ANOVA F-statistic: 227.954, p-value: 0.0000
Kruskal-Wallis H-statistic: 1335.373, p-value: 0.0000


<ipython-input-12-b9a08f845a56>:5: DeprecationWarning: Please import `f_oneway` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  f_stat, p_value = stats.f_oneway(*groups)
<ipython-input-12-b9a08f845a56>:10: DeprecationWarning: Please import `kruskal` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  h_stat, p_kw = stats.kruskal(*groups)


### Ex 3:

Using the winequality_red.csv file in the examples folder of the github repository, test which wine
parameters discriminate the best between wine quality scores categorized into two classes using value
5 as the threshold value (quality>5=“good” and quality<5=“bad”).

In [13]:
wdf = pd.read_csv ('winequality_red.csv', sep= ",")
wdf

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [14]:
# Categorize quality
wdf['quality_category'] = wdf['quality'].apply(lambda x: 'good' if x > 5 else 'bad')

# Drop the original quality column as we created categories
wdf = wdf.drop('quality', axis=1)

# Separate the data by quality category
good_quality = wdf[wdf['quality_category'] == 'good']
bad_quality = wdf[wdf['quality_category'] == 'bad']

# Perform independent t-tests for each parameter (excluding the category column)
parameters = wdf.columns.tolist()
parameters.remove('quality_category')

results = []
for param in parameters:
  # Check if the parameter contains only one unique value in either group
  if good_quality[param].nunique() <= 1 or bad_quality[param].nunique() <= 1:
    results.append({'parameter': param, 't_statistic': 'N/A', 'p_value': 'N/A', 'significant': False, 'reason': 'One group has constant value'})
  else:
    # Perform the t-test
    t_stat, p_val = stats.ttest_ind(good_quality[param], bad_quality[param], equal_var=False, nan_policy='omit') # Using Welch's t-test due to potentially unequal variances
    significant = p_val < 0.05
    results.append({'parameter': param, 't_statistic': t_stat, 'p_value': p_val, 'significant': significant, 'reason': ''})

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)

# Sort by p-value to find the best discriminators (lowest p-value)
results_df = results_df.sort_values(by='p_value', ascending=True)

print("Results of independent t-tests comparing 'good' vs 'bad' quality wine parameters:")
print(results_df)

# Identify parameters that significantly discriminate
discriminating_parameters = results_df[results_df['significant'] == True]

print("\nParameters that significantly discriminate between good and bad quality wine (p < 0.05):")
discriminating_parameters

Results of independent t-tests comparing 'good' vs 'bad' quality wine parameters:
               parameter  t_statistic       p_value  significant reason
10               alcohol    19.782185  1.139485e-77         True       
1       volatile acidity   -13.477521  3.480735e-39         True       
6   total sulfur dioxide    -9.339911  3.822617e-20         True       
9              sulphates     8.853056  2.357718e-18         True       
7                density    -6.554125  7.561697e-11         True       
2            citric acid     6.479900  1.220359e-10         True       
4              chlorides    -4.286526  1.952537e-05         True       
0          fixed acidity     3.862248  1.168400e-04         True       
5    free sulfur dioxide    -2.458755  1.405254e-02         True       
8                     pH    -0.130453  8.962250e-01        False       
3         residual sugar    -0.086470  9.311035e-01        False       

Parameters that significantly discriminate between go

<ipython-input-14-b00b5c2b88b5>:22: DeprecationWarning: Please import `ttest_ind` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  t_stat, p_val = stats.ttest_ind(good_quality[param], bad_quality[param], equal_var=False, nan_policy='omit') # Using Welch's t-test due to potentially unequal variances


,parameter,t_statistic,p_value,significant,reason
10,alcohol,19.782185,1.139485e-77,True,
1,volatile acidity,-13.477521,3.480735e-39,True,
6,total sulfur dioxide,-9.339911,3.822617e-20,True,
9,sulphates,8.853056,2.357718e-18,True,
7,density,-6.554125,7.561697e-11,True,
2,citric acid,6.479900,1.220359e-10,True,
4,chlorides,-4.286526,1.952537e-05,True,
0,fixed acidity,3.862248,1.168400e-04,True,
5,free sulfur dioxide,-2.458755,1.405254e-02,True,
